# Task 1:

You want to find a person who is well connected to multiple companies in a domain e.g. "Software Development". So that you can connect with them for mentorship and utilize their network for more opportunities in that domain

Some examples of domains: <br>
1. Software Development
2. Computer Software
3. Internet
4. Defense
5. Food

In [1]:
import sys
import os

src_path = '..' # This should be path to parent directory of 'src/utils' 
sys.path.append(src_path)

In [2]:
from creds import *
from src.utils import neo4j_utils, mongo_utils, sql_utils

In [3]:
domain = 'Software'
query = f'''
match (c:Company)-[:WORKS_IN]->(i:Industry)
where i.name =~ '(?i).*{domain}.*'
match (p:People)-[e:EMPLOYEE_IN]->(c)
return p, collect(e) as exp, collect(c) as comps
order by size(exp) desc limit 3; 
'''
# (?i) for case-insensitive

print(f"Running query: \n {query}")
neoCon = neo4j_utils.neo_connector(creds=creds['neo_creds'])
result = neoCon.runQuery(query)
# result

Running query: 
 
match (c:Company)-[:WORKS_IN]->(i:Industry)
where i.name =~ '(?i).*Software.*'
match (p:People)-[e:EMPLOYEE_IN]->(c)
return p, collect(e) as exp, collect(c) as comps
order by size(exp) desc limit 3; 



In [23]:
public_identifiers = ["'"+x['p']['public_identifier']+"'" for x in result]
public_identifiers

["'aakash-chaudhary'", "'aaditijoag'", "'aarazakariaei'"]

We can get extra information about these people from mongoDB and postgres

In [24]:
# From PostgreSQL
sql_query = f'''
select full_name, occupation, headline, summary, city, country from uspeople_pc where public_identifier in ({','.join(public_identifiers)});
'''
pgsql = sql_utils.PostgreSQL_Connector(creds['sql_creds'])
people_details = pgsql.runQuery(sql_query)
people_details

,full_name,occupation,headline,summary,city,country
0,Aaditi Joag,Software Development Engineer II at Amazon,Software Development Engineer at Amazon,None,San Francisco Bay Area,US
1,Aakash Chaudhary,Software Development Engineer 3 at Expedia Group,SDE 3 at Expedia Group | Former SDE at Amazon ...,Expert Software Development professional with ...,Seattle,US
2,Aara Zakariaei,Healthcare Account Executive at Slack,Account Executive at Slack | NASM Health Coach,None,Denver,US


## Result:
These people have most experience in the specified domain. We have retrieved their profile summaries also so you can get to know about them and try to connect to them for mentorship and come in direct contact with most companies in your specified domain